# Easy monitoring example

Ronald Tabernig, William Albert, Hannah Weiser, July 2025 

This notebook demonstrates the data preparation. The dataset contains **6 scans** of a [rockfall area in Trier, Germany](https://www.youtube.com/watch?v=bOp4XN9FM48) captured hourly in the night from 25 August 2024 to 26 August 2024.

## Imports

In [ ]:
import py4dgeo
import laspy
import numpy as np
import os
import sys
import json
sys.path.insert(0, "../src")
from fourdgeo import projection
from fourdgeo import utilities
from fourdgeo import change

# File download and handling
from pathlib import Path
import pooch
from tqdm import tqdm

# Image handling
from PIL import Image, ImageDraw, ImageFont, ImageSequence
from IPython.display import HTML

## Get the data

In [2]:
# Handle file download/reading here
data_url = "https://heibox.uni-heidelberg.de/f/6c2a4e6755b74d1abad0/?dl=1"
data_hash = "77b343183c86cbc3f72e0edbe362cc3219d41e00fcf4389ab650a09a13b1b1ec"
file_name = "rockfall_trier.zip"
data_folder = "data/rockfall_trier"

if not Path(data_folder).exists():
    fnames = pooch.retrieve(url=data_url,
                            known_hash=data_hash,
                            path="./",
                            fname=file_name,
                            processor=pooch.Unzip(extract_dir=data_folder),
                            progressbar=True)
    os.remove(file_name)

## Projections
### Prepare the configuration file

We use a configuration dictionary that contains general project settings like the `output_folder` and the relevant settings for the point cloud projection. For the projection, parameters like the `camera_position` and the `resolution_cm` are essential.

In [6]:
configuration = {
    "project_setting": {
        "project_name": "Rockfall_monitoring",
        "output_folder": "./out/rockfall_monitoring",
        "temporal_format": "%y%m%d_%H%M%S",
        "silent_mode": True,
        "include_timestamp": False
    },
    "pc_projection": {
        "pc_path": "",
        "make_range_image": True,
        "make_color_image": False,
        "top_view": False,
        "save_rot_pc": False,
        "resolution_cm": 12.5,
        "camera_position": [
            0.0,
            0.0,
            0.0
        ],
        "rgb_light_intensity": 100,
        "range_light_intensity": 10,
        "epsg": None
    }
}

### Generate the background images

We now generate the background images. For this, we are using classes and functions from the `fourdgeo` library. The class `PCloudProjection` directly takes our configuration file as input and writes them into our specified `output_folder`.

In [ ]:
observations = {"observations": []}

laz_paths = list(Path(data_folder).glob("*.laz"))
laz_paths = sorted(laz_paths)

images = []
list_background_projections = []
pcs = sorted(laz_paths)

for enum, pc in enumerate(pcs):
    lf = laspy.read(pc)
    configuration['pc_projection']['pc_path'] = pc
    project_name = configuration['project_setting']['project_name']
    output_folder = configuration['project_setting']['output_folder']

    background_projection = projection.PCloudProjection(
        configuration = configuration,
        project_name = project_name,
        projected_image_folder = output_folder,
    )
    # First projection
    if enum == 0:
        (
            ref_h_fov, ref_v_fov, ref_anchor_point_xyz, 
            ref_h_img_res, ref_v_img_res
        ) = background_projection.project_pc(buffer_m = 0.5)
    # Next projections using reference data
    else:
        background_projection.project_pc(
            ref_theta=ref_h_fov[0],
            ref_phi=ref_v_fov[0],
            ref_anchor_point_xyz=None,
            ref_h_fov=ref_h_fov,
            ref_v_fov=ref_v_fov,
            ref_h_img_res=ref_h_img_res,
            ref_v_img_res=ref_v_img_res
        )

    bg_img = background_projection.bg_image_filename[0]
    if bg_img[0] == ".":
        bg_img = bg_img[2:]

    outfile = bg_img.split('.')[0] + f"_{enum}_{enum+1}." + bg_img.split('.')[1]

    try:
        os.rename(bg_img, outfile)
    except FileExistsError:
        os.remove(outfile)
        os.rename(bg_img, outfile)

    images.append(outfile)

    background_projection.bg_image_filename[0] = outfile
    list_background_projections.append(background_projection)

### Convert .tif Images to .png Images

Because the dashboard (currently) does not support TIF files, we need to convert the generated background images to the PNG format.

In [8]:
png_images = []

for background_projection in list_background_projections:
    image_path = background_projection.bg_image_filename[0]
    filename = str.split(image_path, ".")[0]
    try:
        with Image.open(image_path) as im:
            for i, page in enumerate(ImageSequence.Iterator(im)):
                out_path = filename + ".png"
                if not os.path.isfile(out_path):
                    try:
                        page.save(out_path)
                    except:
                        print(out_path)
            png_images.append(out_path)
    except:
        print(filename)

### Project the change events onto the image background

Here, we also project the change events onto the background image using the `ProjectChange` class. The `observation` GeoJSON files are written to the `output_folder`.

In [9]:
project_name = configuration["project_setting"]["project_name"]
list_observation_projection = []

for epoch_id, observation in enumerate(observations['observations']):
    background_projection = list_background_projections[epoch_id]
    observation_projection = projection.ProjectChange(observation=observation,
                            project_name=f"{project_name}_{epoch_id}_{epoch_id+1}",
                            projected_image_path=background_projection.bg_image_filename[0],
                            projected_events_folder=output_folder,
                            epsg=None)

    if observation["geoObjects"] is not None:
        observation_projection.project_change()
    list_observation_projection.append(observation_projection)

### Display the rockfall event in the study site, as soon as detected

We generate a GIF of the time series and display the rockfall event using the polygon outlines as soon as it is detected.

In [10]:
frames = []
gif_path = "../docs/img/rockfall_projections_plus_observations.gif"
font = ImageFont.load_default(size = 30)

for enum, img in enumerate(images[1:]):
    frm = Image.open(img).convert("RGB")
    draw = ImageDraw.Draw(frm)
    draw.text((500, 36), f"Epoch: {enum}", fill=(255, 255, 255), font=font)

    observation_projection = list_observation_projection[enum]

    # Load geojson
    if observation_projection.observation["geoObjects"] is not None:
        with open(observation_projection.geojson_name, 'r') as f:
            geojson_data = json.load(f)

        for feature in geojson_data["features"]:
            coords = np.array(feature["geometry"]['coordinates'][0])
            coords[:,1] *= -1
            coords = coords.reshape(len(coords)*2)
            draw.polygon(list(coords), outline='yellow', width=4)

    frames.append(frm)
    

frames[0].save(
    gif_path,
    save_all=True,
    append_images=frames[1:],
    duration=1000,
    loop=0
)

<img src="img/rockfall_projections_plus_observations.gif" style="width:900px;">

## Extract the final JSON 

In order to include this data into the dashboard, we now need to convert the created geojson data to the dashboard data model. For this, we iterate through all geojson files and create their observation objects and the aggregate them into one final data model object and store it in the `output_folder`. This files can then be loaded with the 4DGeo Dashboard.

Note: The path to the background image files has to be a working url. In this example, we use the temporary hosted localhost, explained in [this section](#visualise-the-data-in-the-dashboard). Later this can and should be replaced by an actual server containing the files.

In [11]:
aggregated_data = utilities.DataModel([])

for (i, observation_projection) in enumerate(list_observation_projection):
    if observation_projection is None:
        continue
    elif observation_projection.observation["geoObjects"] is None:
        img_size = Image.open(png_images[i + 1]).convert("RGB").size
        aggregated_data.observations.append(utilities.Observation(
            startDateTime=observation_projection.observation["startDateTime"],
            endDateTime=observation_projection.observation["endDateTime"],
            geoObjects=[],
            backgroundImageData=utilities.ImageData(
                url=str("http://localhost:8001/" + png_images[i + 1]).replace("\\", "/"),
                width=img_size[0],
                height=img_size[1]
            )
        ))
        continue
    
    with open(observation_projection.geojson_name, 'r') as f:
        geojson_data = json.load(f)
    
    img_size = Image.open(png_images[i + 1]).convert("RGB").size
    geometry = geojson_data.get("features")[0].get("geometry")
    coords = geometry.get("coordinates")
    new_observations = utilities.convert_geojson_to_datamodel(
        geojson=geojson_data,
        bg_img=str("http://localhost:8001/" + png_images[i + 1]).replace("\\", "/"),
        width=img_size[0],
        height=img_size[1]
    )

    aggregated_data.observations.extend(new_observations.observations)

with open(f"{output_folder}/final_data_model.json", "w") as f:
    f.write(aggregated_data.toJSON())

### Visualise the data in the dashboard

To see our results in the actual dashboard, we need to host the created json file to make it accessbile. As a quick and easy solution, we can use the **http.server** python library. The python script `server-host.py` hosts all the files in this directory. So in order to setup this local hosting, we need navigate to the `4DGeo/docs` folder to execute the following command in a commandline:

`python server_host.py`

Lastly, inside of the dashboard, set the data source to `http://localhost:8001/out/rockfall_monitoring/final_data_model.json`.